In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import dash

In [2]:
import plotly
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='Brybtb', api_key='50jVIE7Kj5EttwIAtSyJ')
from plotly import __version__
import plotly.plotly as py

TODO

1. Directory structure for raw inputs, company logos, etc
"scripts/load_data.py"
"in/S&P_500_Researcher_Dataset_1979_-_2018.xlsx"
"in/S&P_500_Composition_Changes_1963_-_2018 (1).xlsx"
"out/market_cap_results.csv"
OR
"db/market_cap_data.sql" (if you load result to sqlite)


In [26]:
# Read inputs into pandas
market_cap_stg = pd.read_excel('S&P_500_Researcher_Dataset_1979_-_2018.xlsx', 
                               sheet_name='Market Capitalizations')
sector_stg = pd.read_excel('S&P_500_Composition_Changes_1963_-_2018 (1).xlsx', 
                           sheet_name='Sector Information')

In [27]:
market_cap_stg.head()

,Ticker,Company Name,1979-03-31 00:00:00,1979-06-30 00:00:00,1979-09-30 00:00:00,1979-12-31 00:00:00,1980-03-31 00:00:00,1980-06-30 00:00:00,1980-09-30 00:00:00,1980-12-31 00:00:00,...,2017-12-31 00:00:00,2018-01-31 00:00:00,2018-02-28 00:00:00,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00
0,A,Agilent Technologies Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21564.34000,23715.17309,22152.03217,21606.22470,21165.84762,19935.94896,19910.19192,21066.76000,21545.26000,22502.26000
1,AA,Alcoa Inc,1909.11866,1804.77388,2070.04416,1930.1296,2060.12775,2092.73375,2600.14,2178.46279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAL,American Airlines Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24740.68124,25829.59456,25796.30900,24285.58440,20065.05270,20350.16060,17480.65592,18208.24908,18641.12096,19032.54766
3,AAL (Alex),Alexander & Alexander,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAP,Advance Auto Parts Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7370.67984,8649.77264,8447.18800,8773.52985,8470.10115,9518.78034,10042.74990,10452.00861,12139.36821,12457.59831


In [28]:
# Melt each dataset so Periods are expressed in rows vs. columns
def clean_df(df, value_name):
    period_colums = [x for x in df.columns if x not in ['Ticker','Company Name']]
    result = pd.melt(df, 
                     id_vars=['Ticker','Company Name'], 
                     value_vars=period_colums, 
                     var_name='Period',
                     value_name=value_name)

    result['Period'] = pd.to_datetime(result['Period'], format='%Y-%m-%d')
    return result[result.Period.dt.month == 12]

In [29]:
market_cap = clean_df(market_cap_stg, 'MarketCap')

In [30]:
sector = clean_df(sector_stg, 'Sector')

In [31]:
# merge sector and market cap info into single df
result = market_cap.merge(sector, how='left', on=['Ticker','Period','Company Name'])
result['Year'] = result['Period'].dt.year
result = result[['Sector','Year','MarketCap','Ticker','Company Name']]

# find max company (by MarketCap) for each sector/year
max_group = result.groupby(['Sector','Year'])
max_result = max_group['MarketCap'].transform(max) == result['MarketCap']
final_result = result[max_result][['Sector','Year','MarketCap','Company Name','Ticker']]

In [32]:
final_result.head()

,Sector,Year,MarketCap,Company Name,Ticker
127,Financials,1979,4056.58134,BankAmerica Corp. (Old),BAC (Old)
512,Industrials,1979,11535.48857,General Electric,GE
534,Consumer Discretionary,1979,14468.20000,General Motors,GM
627,Information Technology,1979,37571.84610,IBM Corp.,IBM
825,Health Care,1979,5439.05225,Merck & Co.,MRK


In [ ]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

import pandas as pd

df = final_result.copy()
df['RelativeSize'] = (df['MarketCap'] / df.MarketCap.max()) 
xrange = df[['Year','MarketCap']].groupby('Year').sum().reset_index()


sectors = df.Sector.unique().tolist()

init_notebook_mode(connected=True)

years = df.Year.unique()

# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
#figure['layout']['xaxis'] = {'title': 'Sector'}
#figure['layout']['yaxis'] = {'range': [df.MarketCap.min(),df.MarketCap.max()], 'title': 'Market Cap'}
figure['layout']['yaxis'] = {'range': [df.RelativeSize.min(),df.RelativeSize.max()], 'title': 'Relative Size'}
figure['layout']['xaxis'] = {'range': [xrange.MarketCap.min(),xrange.MarketCap.max()],'title': 'Market Cap'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1979',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = 1979
for sector in sectors:
    dataset_by_year = df[df['Year'] == year]
    dataset_by_year_and_cont = df[df['Sector'] == sector]
    
    data_dict = {
        'x': list(dataset_by_year_and_cont['MarketCap']),
        'y': list(dataset_by_year_and_cont['RelativeSize']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cont['Company Name']),
        'marker': dict(size=df['MarketCap']/100),
        #'marker': {
        #    'sizemode': 'area',
        #    'sizeref': 1,
        #    'size': list(dataset_by_year_and_cont['MarketCap'])
        #},
        'name': sector
    }
    figure['data'].append(data_dict)
    
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for sector in sectors:
        dataset_by_year_and_cont = df[(df['Year'] == int(year)) & (df['Sector'] == sector)]
        data_dict = {
            'x': list(dataset_by_year_and_cont['MarketCap']),
            'y': list(dataset_by_year_and_cont['RelativeSize']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cont['Company Name']),
            'marker': dict(size=df['MarketCap']/100),
            #'marker': {
            #    'sizemode': 'area',
            #    'sizeref': 1,
            #    'size': list(dataset_by_year_and_cont['MarketCap'])
            #},
            'name': sector
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

iplot(figure)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
plotly.__version__
data = [go.Bar(x=final_result.Year,
            y=final_result.MarketCap)]
py.iplot(data, filename='jupyter-basic_bar')

In [ ]:
import plotly.figure_factory as ff
from __future__ import unicode_literals
y = final_result.fillna('')
#y['Year'] = y['Year'].str.encode('utf-8')
table = ff.create_table(y)
py.iplot(table)


In [ ]:
pd_cols = [x for x in sector.columns if x not in ['Ticker','Company Name']]

y = pd.melt(sector, id_vars=['Ticker','Company Name'], value_vars=pd_cols, var_name='Period')
y['Period'] = pd.to_datetime(y['Period'], format='%Y-%m-%d')
y = y[y['Period'].dt.month == 12]
y.rename(columns={'value':'Sector'}, inplace=True)

In [ ]:
pd_cols = [x for x in mkt_cap_raw.columns if x not in ['Ticker','Company Name']]
x = pd.melt(mkt_cap_raw, id_vars=['Ticker','Company Name'], value_vars=pd_cols, var_name='Period')
x['Period'] = pd.to_datetime(x['Period'], format='%Y-%m-%d')
x = x[x['Period'].dt.month == 12]
x.rename(columns={'value':'MarketCap'}, inplace=True)

In [ ]:
result = y.merge(x, how='left', on=['Ticker','Period','Company Name'])
result['Year'] = result['Period'].dt.year
result.head()

In [ ]:
max_result = result[['Sector','Year','MarketCap','Ticker']].groupby(['Sector','Year','Ticker'])['MarketCap'].transform(max) == result['MarketCap']
final_result = result[max_result][['Sector','Year','MarketCap','Company Name','Ticker']]
#final_result[final_result.Sector == 'Information Technology']

In [ ]:
final_result.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
display = final_result[final_result.Sector == 'Information Technology'][['Year','Company Name','MarketCap']]
display.plot(x='Year',y='MarketCap',legend=True)



In [ ]:
mkt_cap_raw = mkt_cap_raw.drop(columns='Company Name')

In [ ]:
mkt_cap_raw.fillna(0)

In [ ]:
tickers = list(mkt_cap_raw['Ticker'])

In [ ]:
mkt_group = mkt_cap_raw.groupby('Ticker').fillna(0)

In [ ]:
mkt_time = mkt_group.T

In [ ]:
mkt_time.columns = tickers


In [ ]:
test = mkt_time.ix[:, mkt_time.max().sort_values(ascending=False).index]

In [ ]:
test.head(3)

In [ ]:
mkt_cap_plot_raw = test.plot(figsize=(15,6))

# Tests from FFN

In [ ]:
test.head()

In [ ]:
data = [go.Scatter(x=test.index,
                  y=test['MSFT'],
                  text=test['MSFT'],
                  mode='markers',
                  marker=dict(size=10,
                  color=test['MSFT'],
                  showscale=True))]

layout=go.Layout(title='If this works?!?!')

fig = go.Figure(data=data, layout=layout)

In [ ]:
py.iplot(fig)

In [ ]:
trace0 = [go.Scatter(x=test.index,
                  y=test['GE'],
                  text=test['GE'],
                  mode='markers',
                  marker=dict(size=10,
                  color=test['GE'],
                  showscale=True))]

In [ ]:
trace1 = [go.Scatter(x=test.index,
                  y=test['GE'],
                  text=test['MSFT'],
                  mode='markers',
                  marker=dict(size=10,
                  color=test['MSFT'],
                  showscale=True))]

In [ ]:
data=[trace0, trace1]
layout=go.Layout(title='If this works?!?!')

fig = go.Figure(data=data, layout=layout)

In [ ]:
import ffn
#%pylab inline

In [ ]:
from iexfinance.stocks import Stock
tsla = Stock('TSLA')
tsla.get_price()

In [ ]:
from iexfinance.stocks import Stock
general_dynamics = Stock('GD')
gd_financials = general_dynamics.get_financials()
gd_financials

In [ ]:
list_of_cash_change = [col for col in gd_financials.columns if col.startswith('cashChange')]

In [ ]:
# gd_financials = general_dynamics.get_financials(format='pandas')
gd_financials = general_dynamics.get_financials(format='json')

In [ ]:
gd_financials = pd.DataFrame(gd_financials)


In [ ]:
gd_financials.columns

In [ ]:
list_of_cash_change = [col for col in gd_financials.columns if col.startswith('currentDebt')]

In [ ]:
df2 = gd_financials[list_of_cash_change]

In [ ]:
df2.columns

In [ ]:
industrials = Stock('BA')

In [ ]:
industrials_financials = industrials.get_financials(format='pandas')


In [ ]:
industrials_df = pd.DataFrame(industrials_financials)
industrials_df.info()

In [ ]:
net_income = [col for col in industrials_df.columns if col.startswith('netIncome')]

In [ ]:
df3 = industrials_df[net_income]

In [ ]:
data = [go.Scatter(x=df2.columns,
                   y=df2.iloc[0:],
                   mode='lines')]

In [ ]:
py.iplot(data)